In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Reconstructing a dataset from the Diamond Light Source facility

Diamond Light Source (DLS) is the UK’s national synchrotron science facility, located at the Harwell Science and Innovation Campus in Oxfordshire.

This exercise will walk you through the reconstruction of a parallel beam 3D data set acquired by DLS. In this set-up parallel beams of X-rays are emitted on to a 2D detector array which allows us to reconstruct a 3D volume. This common geometry for data from a synchrotron source is shown below. 

<img src="figures/parallel3d.png" width=600 height=600 align="centre">

**Learning objectives:**
1. You will be able to read in a data set and manipulate it in to the form required for the ASTRA projectors
2. Use CIL processors CenterOfRotation() and Resizer() to pre-process the data
3. Apply the same reconstruction alorithms to real data that we previously have to simulated data

First, all required imports are carried out. As before this includes tools from the ccpi.framework and ccpi.optimisation modules, but now we also use tools from the ccpi.processors and ccpi.io modules.

The ASTRA projectors are imported from ccpi.astra.oprators and the ccpi.astra.processors modules.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ccpi.framework import ImageData, ImageGeometry
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.framework import BlockDataContainer

from ccpi.framework.TestData import data_dir

from ccpi.optimisation.algorithms import CGLS
from ccpi.optimisation.operators import BlockOperator, Gradient

from ccpi.processors import Resizer, CenterOfRotationFinder

from ccpi.io import NEXUSDataReader

from ccpi.astra.operators import AstraProjectorSimple , AstraProjector3DSimple
from ccpi.astra.processors import FBP

from utilities import islicer, link_islicer
from utilities import plotter2D

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy

### Read in the dataset

Use the NEXUS data reader to read in a dataset from the Diamond Light Source. The data reader creates the AquisitionData object for you with the geometry specified in the file.

CIL also provides a reader for Nikon datasets `NikonDataReader()`.

In [ ]:
## Set up a reader object pointing to the Nexus data set
path = os.path.join(data_dir,'24737_fd_normalised.nxs')
myreader = NEXUSDataReader(nexus_file=path)
data_raw = myreader.load_data()

#Convert the data from intensity to attenuation by taking the negative log
data_raw.log(out=data_raw)
data_raw *= -1

The NEXUSDataReader output is either an ImageData object or an AcquisitonData Object. This is decided by the fields present in the dataset.

We have created an AcquisitionData object from the input file. We can see the raw data has 3-axes where 'vertical' and 'horizontal' describe the detector axes and 'angle' giving the rotation of the object.

In [ ]:
print(type(data_raw))
print(data_raw)

islicer(data_raw, direction='angle', minmax=(0,3), size=8)

### Set up the data ready for ASTRA

In order to use the ASTRA projectors we need to manipulate the data in to the form the ASTRA projectors expect.

In 3D geometry we need the data in the form `['vertical','angle','horizontal']`, which doesn't match the DLS dataset.

In [ ]:
print(data_raw.dimension_labels)

We can use `AcquisitionData.subset()` to return an ordered subset of the AcquisitionData and regenerate the geometry.

This can be used to slice along the dataset in any dimension. For example `data.subset(vertical=n)` will create a new dataset containing the horizontal and angle data at vertical=n

On the other hand, `AcquisitionData.subset(dimensions=['horizontal','vertical','angle'])` will return the same size array with the axes transposed to match the specified order.

<span style="color:red;font-size:larger">**Exercise 1:**</span> Reorder the data axes to prepare the data for the ASTRA operators.

In [ ]:
data = data_raw.subset(dimensions=['vertical','angle','horizontal'])
print(data_raw)
print(data)

ASTRA also requires the of projection angles to be in radians. DLS strores their angular data in degrees so we need to convert them.

In [ ]:
#convert the angles to radians
if data.geometry.angle_unit == 'degree':
    data.geometry.angle_unit = 'radian'
    data.geometry.angles = data.geometry.angles * np.pi /180.

In [ ]:
#look at the acquisition data
islicer(data, direction='angle', minmax=(0,3), size=8)

### Use processors to pre-proccess the data

CIL gives you access to some commonly needed data processors including:
- `Normalizer()` normalises AcquisitionData based on the instrument reading with and without incident photons or neutrons
- `Resizer()` allows you to crop or bin the data in any dimension
- `CenterOfRotationFinder()` finds the center of rotation in a parallel beam dataset (credit: Nghia Vo)

The processors are called in the following way:<br>
>processor_instance = Processor(set_up_parameters)<br>
>processor_instance.set_input(data_in)<br>
>data_out = processor_instance.get_output()<br>

<a id="section_resizer"></a>
#### Resizer()
`Resizer(roi, binning)`

Resizer() is a processor used to crop or bin the data.

To crop the data pass the optional region of interest parameter `roi`. This is a list where each element defines the behaviour along one dimension. To crop along an axis pass a tuple containing the start and end coordinates of the crop `roi=[-1,-1,(index0, index1)]` will crop the data between index0 and index1 in dimension 2.

To bin the data in any dimension pass an optional paramer `binning`. This is a list with the number of pixels to bin in each dimension `binning = [1, 1, 2]` will bin the data in blocks of 2 in dimension 2.


<span style="color:red;font-size:larger">**Exercise 2:**</span> Have you noticed the bad pixel in the top left of each angular projection. Use `Resizer()` to remove the first row of data.

In [ ]:
print(data)

In [ ]:
#define the region of interest here
roi_crop = [(1, data.shape[0]),-1,-1]

In [ ]:
#initialise the processsor
resizer = Resizer(roi=roi_crop)

In [ ]:
#set the input data
resizer.set_input(data)

In [ ]:
#get the output data
data_reduced = resizer.get_output()

Notice that the acquistion geometry has been generated with the new dimensions

In [ ]:
print(data)
print(data_reduced)

In [ ]:
slicer1 = islicer(data, direction='angle', minmax=(0,3), size=8)
slicer2 = islicer(data_reduced, direction='angle', minmax=(0,3), size=8)
link_islicer(slicer1,slicer2)

### Centre of Rotation
In a well aligned CT system the axis of rotation is perpendicular to the X-ray beam and with the rows of detector pixels.

The centre of rotation is the projection of the axis of rotation on to the detector. The reconstruction assumes this is horizontally centred on the detector. An offset introduces blurring and artefacts in the reconstruction.

A top-down view of a centre of rotation offset during acquisition:
<img src="figures/CofR1.png" width=600 height=600 align="centre">

The projection of the rotation axis on to a 2D detector:
<img src="figures/CofR2.png" width=400 height=400 align="centre">

We need to re-proccess the acquisition data to correct it for this offset.

The code below reconstucts one slice of the data. By shifting the acquisition data and looking at the reconstructed slice we can get a feel for what a centre of rotation offset looks like.

<span style="color:red;font-size:larger">**Exercise 3:**</span> Change the value of centre of rotation offset to find the best reconstruction of this slice. What happens if you change the slice number to 80? How about 20?

In [ ]:
#create empty lists containers for the output
title = []
results = []

#pick a slice to reconstruct
slice_num = 67

#create a new dataset that is just single slice of the data
data_slice =  data_reduced.subset(vertical=slice_num)

# Use the acquisition geometry from subset()
ag = data_slice.geometry

# Create image geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h, voxel_num_y=ag.pixel_num_h )

#pick some values of Centre of rotation offset to compare
offset_list = [0,6,6.5,7]

for shift in offset_list:
   
    #translate the acquisition data
    data_shifted = ag.allocate()  
    scipy.ndimage.interpolation.shift(data_slice.as_array(), (0,-shift), output = data_shifted.as_array(), order=1,mode='nearest')
    
    #Perform a fast reconstruction of the slice using FBP
    fbp = FBP(ig, ag, device='gpu')
    fbp.set_input(data_shifted)
    FBP_output = fbp.get_output()  

    #save the results
    title.append("CoR = %s pixels" % shift)
    results.append(FBP_output)

#plot the results    
plotter2D(results,title,fix_range=True, cmap='viridis')

##### Use CenterOfRotationFinder()

We can use the processor `CenterOfRotationFinder()` to locate the centre of rotation in a parallel beam dataset. This processor is based on Nghia Vo's method. https://doi.org/10.1364/OE.22.019078
    
This processor can be applied to 2D or 3D parallel beam geometries. It will return the centre of rotation of the centre slice in pixels. A different slice can be specified by passing the slice index or 'centre' to `set_slice()`.

In [ ]:
# initialise the processsor
cor = CenterOfRotationFinder()

In [ ]:
# set the input data
cor.set_input(data_reduced)

In [ ]:
# get the output data
center_of_rotation = cor.get_output()

In [ ]:
print("Centre of rotation at x = ", center_of_rotation)
shift = (center_of_rotation - data.shape[2]/2)
print("Centre of rotation - detector centre = ", shift, " pixels")

Does this agree with what you found in Excercise 3?

Now we can correct the acquisition data for the centre of rotation offset above. We do this using a scipy function that shifts and interpolates the data. You could also crop or pad the data to correct for the offset. 

In [ ]:
#allocate the memory
data_centred = data_reduced.geometry.allocate()
#use scipy to do a translation and interpolation of each projection image
shifted = scipy.ndimage.interpolation.shift(data_reduced.as_array(), (0,0,-shift), order=3,mode='nearest')
data_centred.fill(shifted)

In [ ]:
#view the data set
islicer(data_centred, direction='angle', size=8,cmap='gray')

<span style="color:red;font-size:larger">**Exercise 4:**</span> Process the corrected data `data_centred` with  CenterOfRotationFinder() and convince yourself it's now close to the centre of the detector.

Remember processors are used as:
>processor_instance = Processor(set_up_parameters)<br>
>processor_instance.set_input(data_in)<br>
>data_out = processor_instance.get_output()<br>

In [ ]:
# initialise the processsor
cor = CenterOfRotationFinder()

In [ ]:
# set the input data
cor.set_input(data_centred)

In [ ]:
#optional: set the slice to run over using set_slice()
cor.set_slice('centre')

In [ ]:
# get the output data
center_of_rotation = cor.get_output()

print("Centre of rotation at x = ", center_of_rotation)
shift = (center_of_rotation - data_centred.shape[2]/2)
print("Centre of rotation - detector centre = ", shift, " pixels")

### Setting up the 3D reconstruction

#### Set up the 3D Acquistion geometry
In the 2D example we used:<br>
`ag = AcquisitionGeometry(geom_type='parallel', angles=angles, pixel_num_h=number_pixels_x)`<br>

For 3D we need to change the dimension description to pass the number of vertical pixels as `pixel_num_v`<br>

However we've been using the acquistion geometry throughout this notebook so we don't need to redefine it.

In [ ]:
# Create Acquisition Geometry
ag = data_centred.geometry.clone()

#### Set up the 3D Image geometry
In the 2D example we used:<br>
`ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)`

For a 3D reconstruction we also need to pass the number of voxels we want in the $z$-direction as `voxel_num_z`. We can also set the voxel size to be equal to the detector pixel size.

In [ ]:
# Create Image Geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_num_y=ag.pixel_num_h, 
                   voxel_num_z=ag.pixel_num_v,
                   voxel_size_x=ag.pixel_size_h,
                   voxel_size_y=ag.pixel_size_h,
                   voxel_size_z=ag.pixel_size_v)

#### Set up the projector

In the 2D example we used the ASTRA projector:<br>
`'AstraProjectorSimple(volume_geometry, sinogram_geometry, device)`

Now we need to use ASTRA's 3D projector (note this projector is GPU only)<br>
`AstraProjector3DSimple(volume_geometry, sinogram_geometry)`

In [ ]:
A = AstraProjector3DSimple(ig, ag)

### Reconstruct using Filtered Back Projection

Reconstruct the data set using the FBP processor from ASTRA

`from ccpi.astra.processors import FBP`

We Run this in the same way as the processors introduced above.

In [ ]:
# initialise the processsor
fbp = FBP(ig, ag, device='gpu')

In [ ]:
# set the input data
fbp.set_input(data_centred)

In [ ]:
# get the output data
FBP_output = fbp.get_output()

In [ ]:
#plot the results
islicer(FBP_output, direction='vertical', size=10, cmap='viridis')

### Reconstruct using Tikhonov with gradient regularisation

Now we have the acquisition data in good shape it's time to set up the reconstruction. We'll once again work through the Tikhonov example, but as we go we'll modify it to work with the 3D dataset.

Recall we are solving:
$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}\tilde{A} u - \tilde{b}\end{Vmatrix}^2_2$$



with, $\tilde{A} = \binom{A}{\alpha L}$ and, $\tilde{b} = \binom{b}{0}$

where,
- $u$ is the unknown image to be solved for

- $A$ is the projection operator

- $\alpha$ is the regularisation parameter

- $L$ is a regularisation operator

- $b$ is the acquired data



set up the block operator $\tilde{b}$

We'll again use the `Gradient()` operator. Its domain is specified by the image geometry so the code doesn't need changing.

In [ ]:
L = Gradient(ig)

alpha = 10
operator_block = BlockOperator(A, alpha * L)

Set up the block data container, $\tilde{b}$

In [ ]:
zero_data = L.range_geometry().allocate(0)
data_block = BlockDataContainer(data_centred,zero_data )

Run CGLS as before, passing the BlockOperator and BlockDataContainer

In [ ]:
# setup CGLS with the Block Operator and Block DataContainer
x_init = ig.allocate(0)      
cgls_tikhonov = CGLS(x_init=x_init, operator=operator_block, data=data_block, update_objective_interval = 10)
cgls_tikhonov.max_iteration = 1000

In [ ]:
# run the algorithm
cgls_tikhonov.run(100)

Display the results as a stack of 2D slices

In [ ]:
CGLS_tikhonov_output = cgls_tikhonov.get_output()

islicer(CGLS_tikhonov_output, direction='vertical', size=10, cmap='viridis')

<span style="color:red;font-size:larger">**Exercise 5:**</span> Try a range of  𝛼  values ranging from very small to very large, visualise the resulting image and central line profiles, and describe the effect of the regularisation parameter choice. Find the $\alpha$  that (visually) gives you the best solution.

To adapt the optimiser to run over the 3D data we only had to update the `ImageGeomerty`, the `AcquisitionGeometry` and the ASTRA projector. 

### Comparision of results

Compare the FBP and Tikhonov reconstruction of the dataset

In [ ]:
#compare the outputs
clim_range=(0,0.11)
slicer1=islicer(CGLS_tikhonov_output, direction=0,minmax=clim_range,title='CGLS', size=10,cmap='viridis')
slicer2=islicer(FBP_output, direction=0,minmax=clim_range,title='FBP', size=10, cmap='viridis')

link_islicer(slicer1,slicer2)

Plot a line profile of Tikhonov and FBP:

In [ ]:
vertical_ind = 67
horizontal_y_ind = 90

plt.plot(CGLS_tikhonov_output.subset(vertical=vertical_ind,horizontal_y=horizontal_y_ind).as_array(),label='CGLS')
plt.plot(FBP_output.subset(vertical=vertical_ind,horizontal_y=horizontal_y_ind).as_array(),label='FBP')
plt.legend()

As can be seen from images and line profiles, Tikhonov with Gradient regularisation allows us to reduce the noise in the reconstruction substantially. However, we may pay a price in terms of blurring the edges.

**Learning objectives:**

After having worked through this notebook, we have now seen how to:

1. Read in a data set and manipulate it in to the form required for the ASTRA projectors
2. Use CIL processors CenterOfRotation() and Resizer() to pre-process the data
3. Apply the same reconstruction alorithms to real data that we previously have to simulated data